In [1]:
!ls

merged.fasta  raw	 result    test_mapped.sam
meta	      reference  segments  Untitled.ipynb


In [42]:
# Confirm the number of cigars vs samtools mapped reads value (987)

inputfile = open("test_mapped.sam", "r")

    
unmapped_count = 0
mapped_count = 0
cs_count = 0

for line in inputfile:
    
    # Header lines start with '@'
    if line[0] is "@":
        continue
    
    fields = line.split()
    
    cigar = fields[5]
    

    if fields[5] is "*": 
        unmapped_count += 1
    else: 
        mapped_count += 1
        #if len(fields) > 20:
            #print (fields[20])
       
    i = 10
    while i < len(fields):
        entry = fields[i]
        if "cs:Z:" in entry: 
            cs_count += 1
            print (entry)
        i += 1
        
print (unmapped_count)
print (mapped_count)
print (cs_count)

cs:Z::4+c:3*tc:5*gc:8-t:4-a:3-tc:7-tg:20-t:4-cgat:4-taca:1*at:2*gc:4+tgt*ct:8-ct:4-ttc*tg:9-a:5*ag:1*ag:5-gccgct*ta:2+at:2*ga*tc:1+cc:3*gc*gt:4*ac:1*at:9-t:2*ga:2*tc:2*at*at*ac:1*tc:2-ata:1-aat:1+c:4*tg*ct:4-act:2+ag:4-t:5-g*ta:2-ttt:4*tc:3*ta:9-attt:17-a*gt:10*ga:5-a:4-c:1-g:2*at:2-g:6-c*at:2-ccc:1*ct:9-tc:1*tc:1*ga:3-ac:2-t:1-tt:6*tc:1*tc*ta:1-tg:1*tc:13-c:1*ct:2*tc:2*tc:1-tc:22*gc:12-ttt:1-a:15*tg:2-ga:5-tgttt:2*ga:3*ta:2*tc:3-a:4*gc:3-t:5-g:4-ttggggcca:3*ga:3*at*ag:1*at:5+ggg:1*ag*tc*ca:3*ga*tg:2*at:11*ga:12*ta:4-gctc:1-g:9*ct:9*ta:5-t:5-t*tc:5+g:3-att*cg:12+t:4-t:1-tgt:4-a:9*cg:2*ag:4-tt:3-ac:1-a:3-a:2-g:4*tc*ca*tg:3-ga:5-ttt*ta:3*ta:2-tg:3*gt:15-tc:2-agt:4-tcc:12*ct:15*tc:5-g:2-ccc:5*cg:8*tg:9-t:2*ag:4-t:1-c:3*tc:2-ca*at:1-t:1*tc:3-t:3-a:2*ct:1*ca:1*tc*tc*at:3*tc:1*at:13-a:8-c:2*ta:1-ca:6+c*ga:11-c:1*ca:6-t*ga:1-g:5*ag:5*tg:25-tctg:14*at:4-acg:1-t:1*tc:6-a:3*tc:3*tc:2*gc:2-gata:2*ct:4*ac*tc*ca*tc:2*ta:5-tc:3+tg:5-cg:5-tc:3*ca:3*ta*ca:1-g:1-t:7*ta:1*ag:2-tc:8-tt:1*ct:29*tg*ga:3-ga

In [39]:
import re

def count_penalties(cs_string):
    mismatches = 0
    insertions = 0
    deletions = 0
    matches = 0
    
    fields = re.split('(\W)', cs_string)
    
    i = 1
    while i < len(fields):
        
        penalty_type = fields[i]
        penalty_info = fields[i+1]
        
        # Mismatch
        if penalty_type is "*":
            mismatches += 1
        # Insertion
        elif penalty_type is "+":
            insertions += 1
        # Deletion
        elif penalty_type is "-":
            deletions += 1
        # Match
        elif penalty_type == ":":
            matches += int(penalty_info)
            #print (penalty_info)
        
        i += 2
        
    return mismatches, insertions, deletions, matches
    

In [44]:
inputfile = open("test_mapped.sam", "r")

total_bases = 0
num_mismatches = 0
num_insertions = 0
num_deletions = 0
num_matches = 0

for line in inputfile:
    
    # Header lines start with '@'
    if line[0] is "@":
        continue
        
    fields = line.split()
    seq = fields[9]

    # Find cs string, if present
    # If not present, the read was not mapped
    cs_string = "None"
    i = 10
    while i < len(fields):
        entry = fields[i]
        if "cs:Z:" in entry: 
            cs_string = entry
            break
        i += 1
    
    # If read is mapped, count bases and penalties
    if cs_string is not "None":
        total_bases += len(seq)
        
        accuracy_info = cs_string.split("cs:Z:")[1]
        
        mis, ins, delet, match = count_penalties(accuracy_info)
        
        num_mismatches += mis
        num_insertions += ins
        num_deletions += delet
        num_matches += match
    
    

# Error Rates
print ("Mismatch Rate: " + str(float(num_mismatches)/float(total_bases)))
print ("Insertion Rate: " + str(float(num_insertions)/float(total_bases)))
print ("Deletion Rate: " + str(float(num_deletions)/float(total_bases)))

# This considers the length of insertions and deletions
print ()
print ("Num Matches: " + str(num_matches))
print ("Total Bases: " + str(total_bases))
print ("Match Rate: " + str(float(num_matches)/float(total_bases)))

# This considers just the amount of insertions and deletions
all_types = num_deletions + num_insertions + num_mismatches + num_matches
match_rate_v2 = num_matches / float(all_types)
print ()
print ("Match Rate V2: " + str(match_rate_v2))


Mismatch Rate: 0.05905077501548554
Insertion Rate: 0.010151082384109198
Deletion Rate: 0.05584826535181907

Num Matches: 2050882
Total Bases: 3927857
Match Rate: 0.5221376440130076

Match Rate V2: 0.8067792236299601


In [45]:
10000000.0 / 3000000000.0 

0.0033333333333333335